In [9]:
from IPython.display import display, HTML

# 출력 텍스트를 스크롤 가능한 창에 담기
def display_scrollable_text(text):
    display(HTML(f"<div style='max-height: 400px; overflow-y: scroll;'>{text}</div>"))

# pymupdf

In [15]:
import fitz  # PyMuPDF를 import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    chunk_temp = splitter.split_text(text)
    
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    
    return chunks


# 사용 예시
file_path = f"/Users/ormor/root/contest/dacon_financial_search/open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf"  # PDF 파일 경로
chunks = process_pdf(file_path)

# 결과 출력
for i, chunk in enumerate(chunks):
    display_scrollable_text(f"Chunk {i+1}:\n{chunk.page_content}\n")

# pymupdf4llm

- 청크 사이즈 조정이 안됨
- 표를 마크다운 형식으로 가져오니 그나마 낫긴함

In [28]:
import os
import json

class Opt:
    def __init__(self):

        # # 모델별 설정 딕셔너리
        # self.model_configs = {
        #     "meta-llama/Meta-Llama-3.1-8B-Instruct":
        #     {
        #         "quantization_config": None,
        #         "torch_dtype": "auto",
        #         "max_token": 256,
        #     },
        #     "rtzr/ko-gemma-2-9b-it": {
        #         "quantization_config": BitsAndBytesConfig(
        #             load_in_4bit=True,
        #             bnb_4bit_use_double_quant=True,
        #             bnb_4bit_quant_type="nf4",
        #             bnb_4bit_compute_dtype=torch.bfloat16
        #         ),
        #         "torch_dtype": "auto",
        #         "max_token": 450
        #     }
        # }
        
        # 선택된 모델
        self.llm_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
        # self.llm_model_config = self.model_configs[self.llm_model]
        self.llm_peft = False
        self.llm_peft_checkpoint = "Meta-Llama-3.1-8B-Instruct-daconfinanace"
        
        self.embed_models = ["intfloat/multilingual-e5-base", "jhgan/ko-sbert-nli", "intfloat/multilingual-e5-large"]
        self.embed_model = self.embed_models[2]
        
        self.pdf_loader = "pymupdf"
        
        self.base_directory = "open/"
        self.train_csv_path = os.path.join(self.base_directory, "train.csv")
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        self.chunk_size = 512
        self.chunk_overlap = 32
        
        self.ensemble = True
        self.bm25_w = 0.5
        self.faiss_w = 0.5
        
        self.is_submit = True
        
        self.eval_sum_mode = False
        
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_model.split('/')[1]}_{self.embed_model.split('/')[1]}_pdf{self.pdf_loader}_chks{self.chunk_size}_chkovp{self.chunk_overlap}_bm25{self.bm25_w}_faiss{self.faiss_w}_mix_submission.csv"
        
        os.makedirs(self.output_dir, exist_ok=True)
        
    def to_json(self):
        return json.dumps(self.__dict__)
    
        
args=Opt()

In [121]:
import pymupdf4llm
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
import re

def process_pdf(file_path):
    md_text = pymupdf4llm.to_markdown(file_path)
    
    # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
    processed_text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', md_text)
    # # 페이지 구분자인 ---- 를 기준으로 페이지를 분리
    # # pages = processed_text.split("-----")
    
    # return processed_text
    
#    return md_text

    # headers_to_split_on = [
    #     ("#", "Header 1"),
    #     ("##", "Header 2"),
    #     ("###", "Header 3"),
    # ]
    # md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    # md_chunks = md_header_splitter.split_text(md_text)
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=args.chunk_size, chunk_overlap=args.chunk_overlap
    # )
    # splits = text_splitter.split_documents(md_chunks)
    
    
    # splits = text_splitter.split_documents(md_chunks)
    
    
    
    chunk_size_tok = 2000  # 토큰의 청크 크기를 설정합니다.
    # 재귀적 문자 텍스트 분할기를 초기화합니다. 토큰 인코더를 사용하여 청크 크기와 중복을 설정합니다.
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size_tok, chunk_overlap=0
    )
    splits = text_splitter.split_text(
        processed_text
    )  # 주어진 텍스트를 분할합니다.
    
    
    
    return splits
    



# 사용 예시
file_path = f"/Users/ormor/root/contest/dacon_financial_search/open/test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf"  # PDF 파일 경로


# chunks = process_pdf(file_path)

# # 결과 출력
# for i, chunk in enumerate(chunks):
#     # display_scrollable_text(f"Chunk {i+1}:\n{chunk.page_content}\n")
#     print(f'Chunk {i+1}:\n{chunk.page_content}\n')


docs_texts = process_pdf(file_path=file_path)    
print(docs_texts)


['#### FIS  # ISSUE & FOCUS  **발행인 박용주   발행처 04637 서울특별시 중구 퇴계로 10(남대문로5가 537) 메트로타워   T 02)6908-8200   F 02)6312-8959**  **작성 심혜인 연구위원   자료제공 기획재정부 재정관리총괄과**   **2023.08.**   ##### 핵심재정사업 성과관리  **1** 들어가며  **2** 재정사업 성과관리제도와 핵심재정사업  **3** 3대 분야 핵심재정사업[(군)]별 주요 내용  **4** 핵심재정사업 성과관리체계 운영 방향  **5** 나가며  \n-----\n **FIS** **ISSUE & FOCUS**  핵심재정사업 성과관리는 올해 처음 도입  된 제도이다. 정부는 국정과제의 조기 성과  ### 핵심재정사업 성과관리  창출을 지원하기 위해 올해 초 국정운영  핵심가치를 반영해 3대 분야, 12대 핵심재정사업(군)을 선정하고, 지난 3월부터 민간 전문가 등으로 구성된 작업반에서 사업별  성과지표·목표, 사업 추진상 장애요인 및 해소 방안, 향후 재정투자 방향 등에 대해 집중 논의해 왔다. 이를 통해 정부는 2027년까지  5년간 국민체감도가 높고 국정비전이 반영된 핵심재정사업(군)에 재정을 중점적으로 투자하고, 예산편성-집행-성과관리의 전(全)  주기에 걸쳐 밀착·집중 관리함으로써 국민이 체감할 수 있는 가시적인 성과를 창출할 예정이다.\n 이 글에서는 금번 신설된 핵심재정사업 성과관리제도에 대해 알아보고, 재정사업 성과관리 기본계획, 성과관리 추진계획, 핵심재  정사업 중간결과 등 정부 발표자료의 주요 내용을 토대로 핵심재정사업 성과관리체계 현황, 12대 핵심재정사업(군)의 주요 내용 등  을 일목요연하게 정리해 보고자 한다. 또한 향후 제도를 성공적으로 안착시키기 위해 핵심재정사업 성과관리체계를 어떠한 방향  으로 운영해 나가는 것이 필요할지 살펴보았다.', '###### 들어가며  ## 01  **ISSUE** **왜 핵심재정사업에 주목해야 하는가?**  *

In [122]:
# 첫 번째 페이지의 내용을 출력
print("첫 번째 페이지 내용:")
print(docs_texts[0])

print(len(docs_texts))


첫 번째 페이지 내용:
#### FIS  # ISSUE & FOCUS  **발행인 박용주   발행처 04637 서울특별시 중구 퇴계로 10(남대문로5가 537) 메트로타워   T 02)6908-8200   F 02)6312-8959**  **작성 심혜인 연구위원   자료제공 기획재정부 재정관리총괄과**   **2023.08.**   ##### 핵심재정사업 성과관리  **1** 들어가며  **2** 재정사업 성과관리제도와 핵심재정사업  **3** 3대 분야 핵심재정사업[(군)]별 주요 내용  **4** 핵심재정사업 성과관리체계 운영 방향  **5** 나가며  
-----
 **FIS** **ISSUE & FOCUS**  핵심재정사업 성과관리는 올해 처음 도입  된 제도이다. 정부는 국정과제의 조기 성과  ### 핵심재정사업 성과관리  창출을 지원하기 위해 올해 초 국정운영  핵심가치를 반영해 3대 분야, 12대 핵심재정사업(군)을 선정하고, 지난 3월부터 민간 전문가 등으로 구성된 작업반에서 사업별  성과지표·목표, 사업 추진상 장애요인 및 해소 방안, 향후 재정투자 방향 등에 대해 집중 논의해 왔다. 이를 통해 정부는 2027년까지  5년간 국민체감도가 높고 국정비전이 반영된 핵심재정사업(군)에 재정을 중점적으로 투자하고, 예산편성-집행-성과관리의 전(全)  주기에 걸쳐 밀착·집중 관리함으로써 국민이 체감할 수 있는 가시적인 성과를 창출할 예정이다.
 이 글에서는 금번 신설된 핵심재정사업 성과관리제도에 대해 알아보고, 재정사업 성과관리 기본계획, 성과관리 추진계획, 핵심재  정사업 중간결과 등 정부 발표자료의 주요 내용을 토대로 핵심재정사업 성과관리체계 현황, 12대 핵심재정사업(군)의 주요 내용 등  을 일목요연하게 정리해 보고자 한다. 또한 향후 제도를 성공적으로 안착시키기 위해 핵심재정사업 성과관리체계를 어떠한 방향  으로 운영해 나가는 것이 필요할지 살펴보았다.
38


In [77]:
import fitz  # pymupdf
import pandas as pd

def extract_text_and_tables_from_pdf(file_path):
    doc = fitz.open(file_path)
    all_texts = []
    all_tables = []

    for page_num in range(len(doc)):
        page = doc[page_num]

        # 텍스트 추출
        text = page.get_text("text")
        all_texts.append(text)

        # 표 추출 (일반적으로 표는 추출하기가 까다로움)
        tables = page.get_text("blocks")
        for block in tables:
            if block[4] == 0:  # 이 블록이 표인 경우
                table_text = block[4]
                table_lines = table_text.split("\n")
                table_data = [line.split() for line in table_lines]
                df = pd.DataFrame(table_data)
                all_tables.append(df)

    return all_texts, all_tables

# 사용 예시
file_path = "/Users/ormor/root/contest/dacon_financial_search/open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf"

texts, tables = extract_text_and_tables_from_pdf(file_path)

# 추출된 텍스트 확인
for i, text in enumerate(texts):
    print(f"Page {i+1} Text:")
    print(text)
    print("\n---\n")

# 추출된 표 확인
for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    display(table)

# pdfplumber

- 표는 잘 가져옴
- fis도 잘 가져옴
- 내용이 좀 섞인듯

In [72]:
import pdfplumber
import pandas as pd

def process_pdf(file_path):
    # PDF 파일 열기
    with pdfplumber.open(file_path) as pdf:
        all_text = []
        all_tables = []
        
        # 각 페이지 처리
        for page_num, page in enumerate(pdf.pages):
            page_text = f"\n--- Page {page_num + 1} ---\n"  # 페이지 구분을 위한 텍스트 추가
            # 텍스트 추출
            text = page.extract_text()
            if text:
                page_text += text
            all_text.append(page_text)
            
            # 표 추출
            tables = page.extract_tables()
            for table in tables:
                # 표를 Pandas DataFrame으로 변환
                df = pd.DataFrame(table[1:], columns=table[0])
                all_tables.append(df)
        
    return all_text, all_tables

# 사용 예시
file_path = "/Users/ormor/root/contest/dacon_financial_search/open/test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf"
text, tables = process_pdf(file_path)

# 결과 출력
print("Extracted Text:")
for page_text in text:
    print(page_text)

# print("\nExtracted Tables:")
# for i, table in enumerate(tables):
#     print(f"Table {i+1}:")
#     display(table)

Extracted Text:

--- Page 1 ---
23-2호
FIS
2023.08.
ISSUE & FOCUS
발행인 박용주 발행처 04637 서울특별시 중구 퇴계로 10(남대문로5가 537) 메트로타워 T 02)6908-8200 F 02)6312-8959
작성 심혜인 연구위원 자료제공 기획재정부 재정관리총괄과
핵심재정사업 성과관리
1 들어가며
2 재정사업 성과관리제도와 핵심재정사업
3 3대 분야 핵심재정사업(군)별 주요 내용
4 핵심재정사업 성과관리체계 운영 방향
5 나가며

--- Page 2 ---
FIS ISSUE & FOCUS
02
핵심재정사업 성과관리 핵심재정사업 성과관리는 올해 처음 도입 재정사업 성과관리제도와 핵심재정사업
된 제도이다. 정부는 국정과제의 조기 성과
창출을 지원하기 위해 올해 초 국정운영
재정사업 성과관리란 예산이 효과적으로 쓰일 수 있도록 재정사업1)의 목표와 성과지표를 사전에 설정하고, 평가를
핵심가치를 반영해 3대 분야, 12대 핵심재정사업(군)을 선정하고, 지난 3월부터 민간 전문가 등으로 구성된 작업반에서 사업별
통해 예산에 환류하는 과정을 의미
성과지표·목표, 사업 추진상 장애요인 및 해소 방안, 향후 재정투자 방향 등에 대해 집중 논의해 왔다. 이를 통해 정부는 2027년까지
5년간 국민체감도가 높고 국정비전이 반영된 핵심재정사업(군)에 재정을 중점적으로 투자하고, 예산편성-집행-성과관리의 전(全) OECD,WorldBank등은2000년대부터프로그램(program)을기준으로예산을나누어성과정보를생산
주기에 걸쳐 밀착·집중 관리함으로써 국민이 체감할 수 있는 가시적인 성과를 창출할 예정이다. 할것,편성과집행,환류에이르는재정의전과정을성과정보를바탕으로관리할것을강조하기시작
이 글에서는 금번 신설된 핵심재정사업 성과관리제도에 대해 알아보고, 재정사업 성과관리 기본계획, 성과관리 추진계획, 핵심재
-1990년대이후체계적인성과관리제도도입에대한필요성이대두되면서미국

# PyPDFium2Loader

In [62]:
!pip install -qU langchain_community


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [67]:
from langchain_community.document_loaders import PyPDFium2Loader

file_path = "/Users/ormor/root/contest/dacon_financial_search/open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf"
loader = PyPDFium2Loader(file_path)

docs = loader.load()
# docs[0]

for i, doc in enumerate(docs):
    print(f"--- Page {i + 1} ---")
    print(doc.page_content[:])  # 첫 500자만 출력
    print("\n")

--- Page 1 ---
사 업 명
혁신창업사업화자금(융자) (5152-301)
1. 사업 코드 정보
구분 기금 소관 실국(기관) 계정 분야 부문
코드 중소벤처기업
창업 및 진흥기금 중소벤처기업부 중소기업정책실
110 118
명칭 산업·중소기업 
및 에너지 창업 및 벤처
구분 프로그램 단위사업 세부사업
코드 5100 5152 301
명칭 창업환경조성 창업기업지원융자(기금) 혁신창업사업화자금(융자)
2. 사업 지원 형태 및 지원율
직접 출자 출연 보조 융자 국고보조율(%) 융자율 (%)
○ ○
3. 지출계획 총괄표
(단위: 백만원, %)
사업명 2022년
결산
2023년 2024년 증감
당초 (B-A) (A) 수정 확정(B) (B-A)/A
혁신창업사업화자
금(융자) 2,300,000 2,230,000 2,330,000 2,007,800 △222,200 △9.96
4. 사업목적
ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의
창업을 활성화하고 고용창출 도모
ㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의
제품화·사업화를 촉진하여 기술기반 중소기업을 육성
5. 사업근거 및 추진경위
ㅇ 법령상 근거 : 중소기업진흥에 관한 법률 제66조, 제67조, 제74조
중소기업창업지원법 제35조



--- Page 2 ---
ㅇ 추진경위 
- 1998. 4 외환위기 이후 창업 활성화 및 고용 창출을 위하여 지원 시작
- 2005. 2 업체별 연간 대출한도를 20억원으로 확대(기존 10억원)
- 2006. 1 지원대상을 업력 5년미만까지 확대(기존 3년미만)
- 2008. 1 부채비율 예외적용 기업을 업력 5년미만까지 확대(기존 3년 미만)
- 2009. 1 지원대상을 업력 7년 미만까지 확대(기존 5년 미만)
- 2010. 3 재창업자금(실패 경영인에 대한 재기지원) 신규 지원
- 2012. 1 지원대상을 업력 5년 미만으로 축소(기존 7년 미만)
청년전용창업자금(만 39세 이하 청년창업자 대상) 신규 지원
-